<a href="https://colab.research.google.com/github/majidsh97/project-2/blob/main/AST_CycleGAN_ebi_hayede.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/inzva/Audio-Style-Transfer.git
!pip install -q tensorflow-io
!pip install librosa==0.8
!pip install tensorflow-gan
!pip install -q -U tensorflow-addons
#sss

In [ ]:
import tensorflow as tf
import numpy as np
import IPython
import matplotlib.pyplot as plt
import os
import tensorflow_io as tfio
from tensorflow.keras import layers
import librosa
import time
import tensorflow_gan as tfgan
from scipy.signal import resample
import tensorflow_addons as tfa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#newvalue = a * value + b. a = (max'-min')/(max-min) and b = max - a * max

def tensor_to_audio(tensor,fs,N_FFT):
  a = np.zeros_like(tensor)
  a = np.exp(tf.transpose(tensor).numpy()) - 1
  
  # This code is supposed to do phase reconstruction
  p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
  for i in range(10):
      S = a * np.exp(1j*p)
      x = librosa.istft(S)
      p = np.angle(librosa.stft(x, N_FFT))
  return x
  OUTPUT_FILENAME =   '1.wav' 
  librosa.write_wav(OUTPUT_FILENAME, x, fs)

def gram(inputs):
  #batch nist age batch mikhay bayad ye b be harf ha va ye 1 be inputs.shape azafe beshe
  gram=0
  gram = tf.matmul(tf.transpose(inputs),inputs)/(inputs.shape[0]*inputs.shape[1])
    
  """  if len(inputs.shape)==4:
    ij=tf.cast(inputs.shape[1]*inputs.shape[2],'float32')
    gram=tf.linalg.einsum('bijk,bijd->bkd',inputs,inputs)/ij
  else:
   if len(inputs.shape)==3:
    i=tf.cast(inputs.shape[1],'float32')
    gram=tf.linalg.einsum('bjk,bjd->bkd',inputs,inputs)/i"""

  return gram

def abs_angle_to_complex(abs,ang):
  ang=tf.complex(0.0,ang)
  ang=tf.math.exp(ang)
  z=tf.complex(abs,0.0)*ang
  return z

def t2a(tensor, frame_length, frame_step, fft_length):
  a=tf.complex(tensor,0.0)
  a=tf.cast(a,tf.complex64)
  a = tf.math.exp(a) - 1
  p = 2 * np.pi* tf.random.uniform(a.shape,dtype=tf.float32) - np.pi
  p=tf.complex(0.0,p)
  for i in range(20):
    S = a * tf.math.exp(p)
    x=tf.signal.inverse_stft(S, frame_length, frame_step, fft_length)
    stft=tf.signal.stft(x,frame_length, frame_step,fft_length)
    p = tf.math.angle(stft)
    p = tf.complex(0.0,p)
  return x

def imshow(img):
  plt.imshow(img+127.5/255.0)

def linear_scale(x,max_to=1,min_to=0):
  max=x.max()
  min=x.min()
  a = (max_to-min_to)/(max-min)
  b = max - a * max
  newvalue = a * x + b  
  return newvalue


def display(x,fs):
  IPython.display.display(IPython.display.Audio(x,rate=fs))

def invspectogram(spect,angle,frame_length,frame_step,fft_length):
  spect = abs_angle_to_complex(tf.math.expm1(spect),angle)
  y_invstft=tf.signal.inverse_stft(spect,frame_length,frame_step,fft_length)
  return y_invstft

def spectrogram(x,frame_length , frame_step,fft_length ):
  y=tf.signal.stft(x,frame_length,frame_step,fft_length)
  content_angle=tf.math.angle(y)
  content_spect=tf.math.log1p(tf.abs(y))
  return [content_spect , content_angle]

def window_stack(a, stepsize, width):
    return tf.stack( [a[i:1+i-width or None:stepsize]  for i in range(0,width) ],-1)

class ClipConstraint(tf.keras.constraints.Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return tf.keras.backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [ ]:
"""a=tfio.audio.AudioIOTensor(FILE)
fs=a.rate.numpy()
a=a.to_tensor()[:,0]
a=tf.cast(a,tf.float32).numpy()
a = a / 32768.0"""

In [ ]:

def get_sliced_spectrom(s,p):
  #x=tf.roll(x,tf.random.uniform((),0,int(n/2),dtype=tf.int32),0)
  #x = window_stack(x,int(fs_content),fs_content)[...,None]

  t = int(s.shape[0]/s.shape[1])*s.shape[1]
  s_slice = np.reshape(s[:t,:],(-1,s.shape[1],s.shape[1]))
  p_slice = np.reshape(p[:t,:],(-1,s.shape[1],s.shape[1]))
  l2=tf.reduce_sum(tf.square(s_slice),[1,2])
  condition=l2>1e4
  s_slice = s_slice[condition,...,None]
  p_slice = p_slice[condition,...,None]

  return tf.concat( [s_slice,p_slice ] ,-1)



In [ ]:
frame_length=1024
frame_step=256
fft_length = None
"""
singer = 'ebi'
DIR='/content/drive/My Drive/music/'+singer+'/'
musiclist=os.listdir(DIR)
data_list=[]
for l in musiclist:
  print(l)
  
  FILE=DIR+l+'/vocals.wav'
  a,fs=librosa.load(FILE)

  s,p = spectrogram(a,frame_length,frame_step,fft_length)
  s=s[:,:s.shape[1]-1]
  p=p[:,:p.shape[1]-1]
  data_list.append(get_sliced_spectrom(s,p))


"""

In [ ]:
hayede_np = np.load('/content/drive/My Drive/music/hayede.npy','r')
ebi_np = np.load('/content/drive/My Drive/music/ebi.npy','r')

BATCH_SIZE=1
hayede = tf.data.Dataset.from_tensor_slices(hayede_np).shuffle(100).batch(BATCH_SIZE,True).prefetch(2)
ebi = tf.data.Dataset.from_tensor_slices(ebi_np).shuffle(100).batch(BATCH_SIZE,True).prefetch(2)


In [ ]:
stride=1
dilation=1
init = tf.keras.initializers.RandomNormal(stddev=0.02)
const = tf.keras.constraints.MaxNorm() #ClipConstraint(0.01)

class WeightedSum(tf.keras.layers.Layer):
  def __init__(self,name):
    super(WeightedSum,self).__init__()
    self.alpha = tf.Variable( tf.abs(init((1,))), trainable=True ,  dtype=tf.float32 , name=name )

  def call(self,inp):
    #return (1-self.alpha) * inp[0] + self.alpha * inp[1]
    return (1-self.alpha) *inp[0] +  self.alpha * inp[1] #+ tf.random.normal(inp[1].shape[1:],0.02)


class discBlock(layers.Layer):
  def __init__(self,f):
    super(discBlock,self).__init__()
    #-------------------------------
    self.c1 = layers.Conv2D(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )
    self.c2 = layers.Conv2D(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )
    #self.bn = layers.BatchNormalization()
    #self.innorm = tfa.layers.normalizations.InstanceNormalization()

    self.lr = layers.LeakyReLU(0.2)
    #self.drop1=layers.Dropout(0.1)
    #-------------------------------
    self.avg = layers.AveragePooling2D()
    #-------------------------------

    
  def call(self,inp):
    x = inp
    x = self.c1(inp)
    x = self.c2(x)
    #x = self.bn(x)
    #x = self.innorm(x)
    x = self.lr(x)
    #x = self.drop1(x)
    x = self.avg(x)
    return x
  
  def plot(self):
    _inp = layers.Input((32,32,3))
    o=self.call(_inp)
    m=tf.keras.Model(_inp,o)
    img = tf.keras.utils.plot_model(m,'discBlock.png',True,dpi=64)
    print("discBlock")
    IPython.display.display(img)
    print('------------------------------')


class genBlock(layers.Layer):
  def __init__(self,f,use_last_layer):
    super(genBlock,self).__init__()
    #-------------------------------
    self.up1 = layers.UpSampling2D()
  #-----------------------------------------
  
    self.c1 = layers.Conv2DTranspose(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )
    self.c2 = layers.Conv2DTranspose(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )
    #self.bn = layers.BatchNormalization()
    self.innorm = tfa.layers.normalizations.InstanceNormalization()
    self.lr = layers.LeakyReLU(0.2)
    self.use_last_layer = use_last_layer
    if self.use_last_layer == True:
      self.last_layer = layers.Conv2DTranspose(1,5,padding='SAME',use_bias=True, kernel_initializer=init , kernel_constraint=const )

    #-------------------------------
    
  def call(self,inp):
    x = inp
    x = self.up1(x)
    x = self.c1(x)
    x = self.c2(x)
    #x = self.bn(x)
    x = self.innorm(x)
    x = self.lr(x)
    if self.use_last_layer == True:
      x = self.last_layer(x)

    return x
    
  def plot(self):
    _inp = layers.Input((32,32,3))
    o=self.call(_inp)
    m=tf.keras.Model(_inp,o)
    img = tf.keras.utils.plot_model(m,'genBlock.png',True,dpi=64)
    print("genBlock")
    IPython.display.display(img)
    print('------------------------------')
  
    

def udown(inp,f):
  x = inp
  f = f * 2
  #x = layers.Conv2D(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )(x)
  #------------------------ bala bayad add she -----------------------------------
  x = discBlock(f)(x)

  return x

def uup(inp,f,use_last_layer):
  x=inp
  f=int(f/2)
  x = genBlock(f,use_last_layer)(x)
  #------------------------ paiin bayad add she -----------------------------------
  #x = layers.Conv2DTranspose(f,5,stride,dilation_rate=dilation,padding='SAME',use_bias=False, kernel_initializer=init , kernel_constraint=const )(x)

  return x

def get_discriminator(inp_shape,num,f):
  inp = layers.Input(inp_shape,dtype=tf.float32)
  x=inp
  layer_list=[]
  for i in range(num):
    x=udown(x,f)
    f=x.shape[-1]
    layer_list.append(x)

  #x = layers.GlobalAveragePooling2D()(x)
  #x = layers.Conv2D(x.shape[-1],4,use_bias=False,kernel_initializer=init)(x)
  #x = layers.Flatten()(x)
  #x = layers.Dense(1,use_bias=True,kernel_initializer=init,kernel_constraint=const)(x)
  return tf.keras.models.Model(inp,x)

def get_generator(input_shape,num,f,latent_dim,is_latent=True):
  inp = layers.Input(input_shape,dtype=tf.float32)
  x = inp
  use_last_layer = False

  if is_latent == True:
    inp = layers.Input((latent_dim,),dtype=tf.float32)
    x = layers.Dense(input_shape[0]*input_shape[1]*input_shape[2],kernel_constraint=const)(x)
    x = layers.Reshape(input_shape)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
  
  
  for i in range(num):
    if i == num-1:
      use_last_layer=True

    x=uup(x,f,use_last_layer)
    f=x.shape[-1]


  return tf.keras.models.Model(inp,x)


gb=genBlock(3,True)
gb.plot()
db=discBlock(3)
db.plot()

In [ ]:
"""def get_filters(f_size):
  filters = [f_size*2**i for i in range(0,6)]
  return filters
"""
#discriminator.trainable = True


In [ ]:
class UentClass():
  def __init__(self,INPUT_SHAPE,NUM,filter_init_size,latent_dim):

    self.discriminator = get_discriminator(INPUT_SHAPE,NUM,filter_init_size)
    self.generator = get_generator(self.discriminator.output_shape[1:],NUM,self.discriminator.output_shape[-1],latent_dim,False)

    #self.plot()

    #_x=_y=int(INPUT_SHAPE[0]/2**num)
    #_z=2**(num+1)
    #generator = get_generator((_h,_w,_z),num,_z)

  def plot(self):
    for m in [self.discriminator , self.generator]:
      print(len(m.layers))
      m.summary()
      img = tf.keras.utils.plot_model(m,dpi=64,show_shapes=True)
      IPython.display.display(img)
  def get_layers(self):

    return zip(self.discriminator.layers[1:] , reversed(self.generator.layers[5:]))
    #print(d.name , d.input_shape,g.name ,g.output_shape)

  def set_trainability(self,flag):
    for d,g in self.get_layers():
      d.trainable=flag
      g.trainable=flag 

  def get_unet(self,g_in=0,d_out=False ):

    x=self.discriminator.input
    q=[]
    #q.append(x)
    for i in range(len(self.discriminator.layers)):
      l = self.discriminator.layers[i]
      if isinstance(l,layers.InputLayer)==True:
        continue

      x=l(x)
      q.append(x)


    q.reverse()
    print(q)
    print(len(q))
    #x = layers.UpSampling2D()(x)
    #x = layers.Concatenate()([x,x])
    #--------------------------------------------------
    #x = layers.Flatten()(x)
    #x = layers.Dense(x.shape[1])(x)
    #x = layers.Reshape(generator.input_shape[1:])(x)
    #--------------------------------------------------
    disc_output = x
    if d_out == True:
      disc_output = layers.Flatten()(disc_output)
      disc_output = layers.Dense(1)(disc_output)
    
    #x = layers.Flatten()(x)
    if g_in==1:
      x = self.generator.input
    if g_in == 2:
      x = WeightedSum('alpha_middle')([x,self.generator.input])


    for i in range(len(self.generator.layers)):
      l = self.generator.layers[i]
      if isinstance(l,layers.InputLayer)==True:
        continue 
      x=l(x)
      if i < len(q):
        print(x,q[i])
        #x = WeightedSum("alpha_"+str(i))([q[i] , x])
        x = layers.Add()([q[i],x])
      

    #x  = layers.Conv2DTranspose(1,5,padding='SAME')(x)
    #x = tf.nn.tanh(x)
    outputs=[]
    inputs=[self.discriminator.input]
    if d_out==True:
      outputs.append(disc_output)

    outputs.append(x)

    if g_in==1 or g_in == 2:
      inputs.append(self.generator.input)

    return tf.keras.Model(inputs,outputs)

"""unetclass=UentClass((512,512,1),4,16,256)
unetclass.plot()
layer_zipped =unetclass.get_layers()
for d,g in layer_zipped:
  print(d.name , d.input_shape,g.name ,g.output_shape)


unet= unetclass.get_unet(0,False)
unet.summary()
tf.keras.utils.plot_model(unet,dpi=64,show_shapes=True)"""

In [ ]:
def get_alphas(_model):
  alphas=[]
  for v in _model.non_trainable_variables:
    if v.name[:5]=='alpha':
      alphas.append(v)
  return alphas

def set_alphas(alphas,new_values):
  for a,nv in zip(alphas,new_values):
    a.assign(nv)

#alphas= get_alphas(unet)
#print(alphas)
#new_values = tf.ones_like(alphas) * 0.2
#set_alphas(alphas , new_values)
#print(alphas)


In [ ]:
filter_init_size=7
latent_dim = 256
NUM = 5
generator_g_class=UentClass((512,512,1),NUM,filter_init_size,latent_dim)
generator_g= generator_g_class.get_unet(0,False)

generator_f_class=UentClass((512,512,1),NUM,filter_init_size,latent_dim)
generator_f= generator_f_class.get_unet(0,False)

generator_g.summary()
tf.keras.utils.plot_model(generator_g,dpi=64,show_shapes=True)

In [ ]:
#discriminator_extended = unetclass.discriminator
discriminator_x = get_discriminator((512,512,1),NUM,filter_init_size)
x = discriminator_x.output
#x = discBlock(x.shape[-1])(x)
x = layers.Flatten()(x)
#x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1,kernel_constraint=const)(x)
discriminator_x = tf.keras.Model(discriminator_x.input , x )


discriminator_y = get_discriminator((512,512,1),NUM,filter_init_size)
x = discriminator_y.output
#x = discBlock(x.shape[-1])(x)
x = layers.Flatten()(x)
#x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1,kernel_constraint=const)(x)
discriminator_y = tf.keras.Model(discriminator_y.input , x )
discriminator_y.summary()
tf.keras.utils.plot_model(discriminator_y,dpi=64,show_shapes=True)

In [ ]:

NUM_SAMPLE=5
x = next(iter(ebi))
x_spect_const= x[...,0:1]
x_angle_const= x[...,1:2]
y = next(iter(hayede))
y_spect_const= y[...,0:1]
y_angle_const= y[...,1:2]

"""x_spect_const= ebi_spect[NUM_SAMPLE][None,...,None]
x_angle_const= ebi_angle[NUM_SAMPLE][None,...,None]

y_spect_const=hayede_spect[NUM_SAMPLE][None,...,None]
y_angle_const=hayede_angle[NUM_SAMPLE][None,...,None]"""

print(x_angle_const.shape)

def result(x_spect,x_angle,y_spect,y_angle):
  y_generated= generator_g(x_spect,training=False)
  x_generated=generator_f(y_spect,training=False)
  #y_generated = tf.nn.relu((y_generated+0.5)*5.0)
  #x_generated = tf.nn.relu((x_generated+0.5)*5.0)

  plt.figure(figsize=(8,8))
  i=0
  for x in [(x_spect, x_angle) , (x_generated,x_angle), (y_spect,y_angle) , (y_generated,y_angle)]:
    spect = x[0][0,:,:,0]
    angle = x[1][0,:,:,0]

    spect = tf.nn.relu((spect+1)*2.5)
    angle = angle * np.pi
    print(np.max(spect))
    plt.subplot(2,2,i+1)
    plt.imshow(spect)
    plt.axis(False)
    a = t2a(spect,frame_length, frame_step, 1023)
    #a = invspectogram(spect,angle,frame_length,frame_step,1023)
    display(a,22050)
    i+=1
  
  plt.show()

result(x_spect_const,x_angle_const,y_spect_const,y_angle_const)

LAMBDA = 10
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

LR =1e-4
generator_g_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.0)
generator_f_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.0)

discriminator_x_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.0)
discriminator_y_optimizer = tf.keras.optimizers.Adam(LR, beta_1=0.0)

@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))
  return [gen_g_loss , gen_f_loss , disc_x_loss , disc_y_loss]

In [ ]:
history=[] 
#alphas= get_alphas(generator)
EPOCHS=15
for e in range(0,EPOCHS):
  start = time.time()   
  
  for x,y in zip(ebi,hayede):
    h = train_step(x[...,0:1],y[...,0:1])
  
  if e%2==1:
    hayede = tf.data.Dataset.from_tensor_slices(hayede_np).shuffle(100).batch(BATCH_SIZE,True).prefetch(2)
    ebi = tf.data.Dataset.from_tensor_slices(ebi_np).shuffle(100).batch(BATCH_SIZE,True).prefetch(2)

  print ('Time for epoch {} is {} sec gen_g_loss {} , gen_f_loss {} , disc_x_loss {} , disc_y_loss {}'.format(e+1, time.time()-start,h[0],h[1] ,h[2],h[3]))
  history.append(h)
  result(x_spect_const,x_angle_const,y_spect_const,y_angle_const)


In [ ]:
plt.figure(figsize=(10,10))
y = np.array(history)
plt.plot(y,'.-')
plt.ylim()#[-1,5])
plt.legend(['loss_g' , 'loss_f' , 'disc_real_loss_x' , 'disc_fake_loss_y' ])
plt.show()


In [ ]:

"""x = next(iter(ebi))
x_spect_const= x[...,0:1]
x_angle_const= x[...,1:2]
y = next(iter(hayede))
y_spect_const= y[...,0:1]
y_angle_const= y[...,1:2]
"""
"""x_spect_const= ebi_spect[NUM_SAMPLE][None,...,None]
x_angle_const= ebi_angle[NUM_SAMPLE][None,...,None]

y_spect_const=hayede_spect[NUM_SAMPLE][None,...,None]
y_angle_const=hayede_angle[NUM_SAMPLE][None,...,None]"""

print(x_angle_const.shape)

def result(x_spect,x_angle,y_spect,y_angle):
  y_generated= generator_g(x_spect,training=False)
  x_generated=generator_f(y_spect,training=False)
  #y_generated = tf.nn.relu((y_generated+0.5)*5.0)
  #x_generated = tf.nn.relu((x_generated+0.5)*5.0)

  plt.figure(figsize=(8,8))
  i=0
  for x in [(x_spect, x_angle) , (x_generated,x_angle), (y_spect,y_angle) , (y_generated,y_angle)]:
    spect = x[0][0,:,:,0]
    angle = x[1][0,:,:,0]

    spect = tf.nn.relu((spect+1)*2.5)
    angle = angle * np.pi
    print(np.max(spect))
    plt.subplot(2,2,i+1)
    plt.imshow(spect)
    plt.axis(False)
    a = t2a(spect,frame_length, frame_step, 1023)
    #a = invspectogram(spect,angle,frame_length,frame_step,1023)
    display(a,22050)
    i+=1
  
  plt.show()

result(x_spect_const,y_angle_const,y_spect_const,x_angle_const)
